In [30]:
using IJulia, ImageIO, ImageMagick, Plots, Images, JLD2, Printf, Random, Statistics, Interact, WebIO

if !@isdefined(g)
    include("src/MyLearn2ClassifyMany.jl")
end
using Main.MyLearn2ClassifyMany

theme(
    :wong;
    label="",
    markerstrokewidth=0.3,
    markerstrokecolor=:white
)

println("Libraries loaded")

Libraries loaded


In [31]:
function e_i(i::Integer, n::Integer, T::Type=Float64)
    v = zeros(T, n)
    v[i] = 1

    return v
end

"""
    img = array2rgb(X)

Given a `height` x `width` x `3` array, return an RGB image with the same
dimensions, where

    img[i, j] = RGB(X[i, j, 1], X[i, j, 2], X[i, j, 3])
"""
function array2rgb(X::Array{T,3}) where T <: Number
    height, width = size(X, 1), size(X, 2)
    img = Array{RGB,2}(undef, height, width)

    for i in 1:height, j in 1:width
        r, g, b = X[i, j, :]
        img[i, j] = RGB(r, g, b)
    end
    return img
end

"""
    Y = array2matrix(X)

Given a four-dimensional array `X`, vectorize each slice of `X` along
its fourth dimension. For example, if `X` has size `height` x `width` x
`rgb` x `n_samples`, then the returned matrix has size
`height * width * rgb` x `n_samples`.
"""
function array2matrix(X::AbstractArray{T,4}) where T
    height, width, rgb, n_samples = size(X)

    Y = zeros(height * width * rgb, n_samples)

    for idx in 1:n_samples
        Y[:, idx] = vec(X[:, :, :, idx])
    end

    return Y
end

"""
    Y = array2matrix(X, slice_dim)

Given an array `X`, vectorize each slice of `X` along dimension `slice_dim`.

This function is useful for converting high-dimensional datasets with a
sample dimension into data matrices for training neural networks.
"""
function array2matrix(X::AbstractArray, slice_dim::Integer)
    return hcat((vec(slice) for slice in eachslice(X; dims=slice_dim))...)
end

function check_correct(f_a, W_hat, X_test, b_hat, n_test)
    Y_out = f_a.(W_hat * X_test .+ b_hat)
    test_predicted = output2labels(Y_out)
    test_actual = [ones(n_test); 2 * ones(n_test); 3 * ones(n_test)]
    pcorrect = length(findall(test_actual .== test_predicted)) / length(test_actual)
    return (
        pcorrect,
        test_predicted,
        test_actual
    )  
end

function output2labels(Y)
    n_samples = size(Y, 2)
    label_idx = [findmax(Y[:, i])[2] for i in 1:n_samples]
end

println("Functions defined")

Functions defined


In [32]:
X_rock = load("data/rock.jld2")["rock_imgs"]
X_paper = load("data/paper.jld2")["paper_imgs"]
X_scissors = load("data/scissors.jld2")["scissors_imgs"]

X = [X_rock, X_paper, X_scissors]
labels = ["rock", "paper", "scissors"]
num_labels = length(labels)

label2vector = Dict{String,Vector{Float64}}()
for (i, l) in enumerate(labels)
    label2vector[l] = e_i(i, num_labels)
end
vector2label = Dict((v, k) for (k, v) in label2vector)

train_fraction = 0.6
n_samples = size(X_rock, 4)
n_train = Int(n_samples * train_fraction)
n_test = n_samples - n_train

X_train = hcat((array2matrix(Xi[:, :, :, 1:n_train]) for Xi in X)...)
Y_train = hcat((repeat(label2vector[l], 1, n_train) for l in labels)...)

X_test = hcat((array2matrix(Xi[:, :, :, (n_train + 1):n_samples]) for Xi in X)...)
Y_test = hcat((repeat(label2vector[l], 1, n_test) for l in labels)...)

println("Training and test data separated")

Training and test data separated


In [33]:
function train(
        f_a,
        df_a,
        grad_loss_1layer,
        X_train,
        Y_train,
        W0,
        b0,
        mu,
        iters,
        batch_size
    )

    println("""μ: $(mu), i: $(iters), B: $(batch_size), f: $(String(Symbol(f_a)))""")

    W_hat, b_hat, loss = learn2classify_asgd_1layer(
        f_a,
        df_a,
        grad_loss_1layer,
        X_train,
        Y_train,
        W0,
        b0,
        mu,
        iters,
        batch_size
    )
    
    loss_end = mean(loss[end-100:end])

    (pcorrect, test_predicted, test_actual) = check_correct(f_a, W_hat, X_test, b_hat, n_test)
    
    println("pc: $(@sprintf("%.2f", pcorrect * 100))%, loss: $(@sprintf("%.2f", loss_end * 100))%")

    loss_plot = scatter(
        loss;
        yscale=:log10,
        ylims=[10^-2,10^0],
        title="""Training loss: $(@sprintf("%.2f", loss_end * 100))%
        Probability correct: $(@sprintf("%.2f", pcorrect * 100))%
        μ: $(mu), i: $(iters), B: $(batch_size), f: $(String(Symbol(f_a)))"""
    )
    
    confusion_matrix = zeros(3, 3)
    for i in 1:3
        indices_i = findall(test_actual .== i)

        # predictions for class i
        predictions_i = test_predicted[indices_i]

        for j in 1:3
            # fraction of class i samples labeled as class j
            confusion_matrix[i, j] = sum(predictions_i .== j) / length(indices_i)
        end
    end
    
    incorrect_idx = findall(test_predicted .!= test_actual)
    correct_idx = findall(test_predicted .== test_actual)

    # should be approximately all ones
    @show sum(confusion_matrix; dims=2)

    confusion_map = heatmap(
        confusion_matrix;
        alpha=1.0,
        grid=false,
        yflip=true,
        xticks=(collect(1:3), ["Rock", "Paper", "Scissors"]),
        yticks=(collect(1:3), ["Rock", "Paper", "Scissors"]),
        aspect_ratio=1.0,
        size=(500, 470)
    )

    params = Dict(
        "mu"=>mu,
        "iters"=>iters,
        "batch_size"=>batch_size,
        "f_a"=>f_a,
        "W_hat"=>W_hat,
        "b_hat"=>b_hat,
        "loss"=>loss,
        "loss_end"=>loss_end,
        "loss_plot"=>loss_plot,
        "pcorrect"=>pcorrect,
        "filename"=>"multiloss_p$(@sprintf("%.2f", pcorrect * 100))%_μ1e$(Int(log10(mu)))_i$(iters)_B$(batch_size)_f$(String(Symbol(f_a))).png",
        "confusion_matrix"=>confusion_matrix,
        "confusion_map"=>confusion_map,
        "incorrect_idx"=>incorrect_idx,
        "correct_idx"=>correct_idx
    )
    
    return params
end

train (generic function with 1 method)

In [34]:
println("Starting training iterations")

getMu(i) = 1/10^i
getIters(i) = Int(1000 * i)
getBatch(i) = Int(10 .* i)
getF(i) = ((linear,dlinear),(tanh,dtanh),(sigmoid,dsigmoid))[i]

d = size(X_train, 1)
W0 = zeros(num_labels, d)
b0 = zeros(num_labels)
plotsdir = "plots"
mkpath(plotsdir)

####### default training values ########

mu = getMu(3) # 1e-5
iters = getIters(4) # 4000
batch_size = getBatch(7) # 40
f_a, df_a = getF(3) # sigmoid, dsigmoid

########################################
# params = []
# for i in 1:20
#     mu = getMu(rand(2:7))
#     f_a, df_a = getF(rand(1:3))
#     batch_size = getBatch(rand(4:8))
#     params = [params; train(f_a,df_a,grad_loss_1layer,X_train,Y_train,W0,b0,mu,iters,batch_size)]
# end

p = train(f_a,df_a,grad_loss_1layer,X_train,Y_train,W0,b0,mu,iters,batch_size)

println("Training complete.")

Starting training iterations
μ: 0.001, i: 4000, B: 70, f: sigmoid
pc: 93.43%, loss: 3.52%
sum(confusion_matrix; dims = 2) = [0.9999999999999999; 1.0; 1.0]
Training complete.


In [35]:
savefig(p["loss_plot"], "$(plotsdir)/$(p["filename"])")
savefig(p["confusion_map"], "$(plotsdir)/conf_$(p["filename"])")

# ps = params
# for i in 1:3
#     val, idx = findmax(collect(p["pcorrect"] for p in ps))
#     p = ps[idx]
#     deleteat!(ps, idx)
#     savefig(p["loss_plot"], p["filename"])
#     display(p["loss_plot"])
# end

In [40]:
h, w, rgb, n_samples = size(X_rock)

println("Incorrectly classified")
@manipulate throttle=0.3 for idx in p["incorrect_idx"]
    X_img = reshape(X_test[:, idx], h, w, rgb)
    
    array2rgb(X_img)
end

Incorrectly classified


Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["idx"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 56,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
28, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/jimmy/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/jimmy/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/jimmy/.julia/packages/InteractBase/sOe2Z/src/../assets/all.js"), Asset("css", nothing, "/home/jimmy/.julia/packages/InteractBase/sOe2Z/src/../assets/style.css"), Asset("css", nothing, "/home/jimmy/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f432c260280, Task (runnable) @0x00007f432c260280), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"12\",\"13\",\"22\",\"56\",\"77\",\"97\",\"135\",\"144\",\"147\",\"166\",\"170\",\"174\",\"183\",\"197\",\"225\",\"252\",\"261\",\"316\",\"324\",\"325\",\"327\",\"332\",\"337\",\"353\",\"377\",\"384\",\"414\",\"434\",\"437\",\"441\",\"450\",\"453\",\"4

In [41]:
outdir = "multiclassified"
try
    rm(outdir, recursive=true)
catch
end
    
for idx in p["incorrect_idx"]
    X_img = reshape(X_test[:, idx], h, w, rgb)
    img = array2rgb(X_img)
    save("$(outdir)/wrong$(idx).png", img)
end

for idx in p["correct_idx"]
    X_img = reshape(X_test[:, idx], h, w, rgb)
    img = array2rgb(X_img)
    save("$(outdir)/right$(idx).png", img)
end